In [9]:
# package imports
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier
# import matplotlib.pyplot as plt 

In [10]:
# load data
aarp = pd.read_csv('Data/clean_AARP.csv')

In [13]:
# list vars
model_vars = ['AGE4_recode', 'INCOME', 'D6_recode', 'D8_recode', 'D9C_recode', 'D18_recode', 'D26_recode', 'MARITAL_recode', 'EDUC4_recode', 'RACETHNICITY_recode', 'GENDER_recode', 'HHSIZE', 'INCOME_recode']

# aarp be aarp
aarp = aarp[model_vars]

aarp['RACETHNICITY_recode'] = pd.Categorical(aarp['RACETHNICITY_recode'])
aarp['GENDER_recode'] = pd.Categorical(aarp['GENDER_recode'])
aarp['MARITAL_recode'] = pd.Categorical(aarp['MARITAL_recode'])
aarp['D26_recode'] = pd.Categorical(aarp['D26_recode'])
aarp['D18_recode'] = pd.Categorical(aarp['D18_recode'])

In [14]:
aarp.dtypes

AGE4_recode               int64
INCOME                   object
D6_recode                 int64
D8_recode                 int64
D9C_recode                int64
D18_recode             category
D26_recode             category
MARITAL_recode         category
EDUC4_recode              int64
RACETHNICITY_recode    category
GENDER_recode          category
HHSIZE                    int64
INCOME_recode             int64
dtype: object